🤟

In [ ]:
!pip install transformers==4.28.0 datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizerFast, DistilBertConfig
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import default_data_collator
from torch.utils.data import DataLoader
from datasets import load_dataset

In [ ]:
# This script selects a random subset of examples from the SQuAD (Stanford Question Answering Dataset) version 2.
# It first sets a seed for the random number generator to ensure reproducibility of results.
import random

seed = 2
random.seed(seed)

# Load the SQuAD version 2 dataset, which contains training examples for question answering tasks.
squad_v2_dataset = load_dataset("squad_v2")

# Determine the total number of examples in the training set.
num_examples = len(squad_v2_dataset["train"])

# Randomly select 1000 indices from the range of available examples.
random_indices = random.sample(range(num_examples), 1000)

# Extract the selected examples from the training dataset based on the randomly chosen indices.
selected_examples = squad_v2_dataset["train"].select(random_indices)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
selected_examples['context'][0]

"The pricing of risk refers to the incremental compensation required by investors for taking on additional risk, which may be measured by interest rates or fees. Several scholars have argued that a lack of transparency about banks' risk exposures prevented markets from correctly pricing risk before the crisis, enabled the mortgage market to grow larger than it otherwise would have, and made the financial crisis far more disruptive than it would have been if risk levels had been disclosed in a straightforward, readily understandable format."

In [ ]:
selected_examples['question'][0]

'What is a measurement of pricing of risk?'

In [ ]:
# Initialize the tokenizer for DistilBERT, a pre-trained transformer-based language model, optimized for fast tokenization.
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
# Load the configuration for DistilBERT, specifying its architecture and model settings.
config = DistilBertConfig.from_pretrained("distilbert-base-uncased")
# Create an instance of DistilBERT for question answering, loading the pre-trained weights based on the specified configuration.
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased", config=config)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [ ]:
def prepare_data(examples):
    # Tokenize the input examples using the pre-defined tokenizer with certain settings.
    # Truncate the input to a maximum length of 512 tokens and slide a window with a stride of 128 tokens.
    # Return overflowed tokens and offset mappings to handle examples that exceed the maximum length.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=512,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding=True,
    )

    # Find the maximum sequence length based on the tokenized examples.
    max_sequence_length = max(len(input_ids) for input_ids in tokenized_examples["input_ids"])
    print("Maximum sequence length:", max_sequence_length)

    # Retrieve the mapping of overflowed tokens to their corresponding original examples.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # Retrieve the offset mapping for each token to its corresponding character in the original context.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Initialize empty lists to store start and end positions of the answers in the tokenized examples.
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    # Iterate through the tokenized examples to map the answer positions to token positions.
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)

        # Find the index of the original example corresponding to the current tokenized example.
        sample_index = sample_mapping[i]

        # Retrieve the answers for the corresponding original example.
        answers = examples["answers"][sample_index]

        # Check if there are no answers for the current example.
        if len(answers["answer_start"]) == 0:
            # If no answers are present, assign the start and end positions to the CLS token index.
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # If there are answers, find the start and end character positions of the answer in the original context.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Find the token positions corresponding to the start and end character positions.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # If the answer spans multiple tokens, adjust the token positions to the boundaries of the answer.
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    # Return the tokenized examples with the mapped start and end positions of the answers.
    return tokenized_examples


In [ ]:
# Map the 'prepare_data' function to the selected examples using the Hugging Face Datasets library.
# This function tokenizes the question and context pairs in batches and prepares the data for question answering.
# The 'batched=True' option indicates that the 'prepare_data' function will be applied in batches for better efficiency.
squad_v2_encoded = selected_examples.map(prepare_data, batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Maximum sequence length: 499


In [ ]:
# Define a custom data collator function to create batches of data for training and evaluation.
# This function takes a list of examples and organizes them into tensors for input_ids, attention_mask,
# start_positions, and end_positions, which are essential for training a question answering model.

def custom_data_collator(batch):
    # Convert the list of input_ids from the batch to a tensor.
    input_ids = torch.tensor([item["input_ids"] for item in batch])

    # Convert the list of attention_mask from the batch to a tensor.
    attention_mask = torch.tensor([item["attention_mask"] for item in batch])

    # Convert the list of start_positions from the batch to a tensor.
    start_positions = torch.tensor([item["start_positions"] for item in batch])

    # Convert the list of end_positions from the batch to a tensor.
    end_positions = torch.tensor([item["end_positions"] for item in batch])

    # Return a dictionary containing the tensors of input_ids, attention_mask, start_positions, and end_positions.
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "start_positions": start_positions,
        "end_positions": end_positions,
    }

# Create a DataLoader for the training dataset 'squad_v2_encoded'.
# Each batch will have a batch size of 8 and will be organized using the custom_data_collator function.
train_dataloader = DataLoader(squad_v2_encoded, batch_size=8, collate_fn=custom_data_collator)

# Create a DataLoader for the evaluation dataset 'squad_v2_encoded'.
# Each batch will have a batch size of 8 and will be organized using the custom_data_collator function.
eval_dataloader = DataLoader(squad_v2_encoded, batch_size=8, collate_fn=custom_data_collator)


In [ ]:
# data_collator = default_data_collator
# train_dataloader = DataLoader(squad_v2_encoded["train"], batch_size=8, collate_fn=data_collator)
# eval_dataloader = DataLoader(squad_v2_encoded["validation"], batch_size=8, collate_fn=data_collator)

In [ ]:
# Define the optimizer for the question answering model using the AdamW algorithm.
# AdamW is a variant of the Adam optimizer that incorporates weight decay regularization.
# The learning rate for the optimizer is set to 3e-4, which is a common learning rate for transformer-based models.
optimizer = AdamW(model.parameters(), lr=3e-4)

# Calculate the total number of training steps based on the number of epochs (3 in this case) and the training data size.
# The total number of training steps is the number of times the model will update its parameters during training.
num_training_steps = len(train_dataloader) * 3  # Number of epochs

# Define the learning rate scheduler using the 'get_linear_schedule_with_warmup' function from Hugging Face Transformers.
# The learning rate scheduler linearly increases the learning rate from 0 to the set value (3e-4) during warm-up steps.
# Here, 'num_warmup_steps' is set to 0, indicating there will be no warm-up phase.
# The learning rate is then linearly decayed from the set value to 0 over the remaining training steps.
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import torch
from torch.utils.data import DataLoader
import random
from datasets import load_dataset

# Function to answer a question given the question, context, model, tokenizer, and device.
def answer_question(question, context, model, tokenizer, device):
    # Tokenize the question and context, convert them to PyTorch tensors, and move them to the device (CPU or GPU).
    inputs = tokenizer(question, context, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Pass the input tensors to the model to get the start and end logits.
    outputs = model(**inputs)

    # Find the start and end positions with the highest logits.
    start_position = torch.argmax(outputs.start_logits)
    end_position = torch.argmax(outputs.end_logits)

    # Convert the token IDs of the answer span to a string answer.
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_position:end_position + 1]))

    # Return the answer string.
    return answer

# Check if a GPU is available and use it; otherwise, use the CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device (CPU or GPU).
model.to(device)

# Training loop for the model over 6 epochs.
for epoch in range(6):
    model.train()
    print(f"Epoch: {epoch + 1}")
    total_loss = 0.0
    total_correct = 0
    total_examples = 0

    # Iterate through the training dataloader to process batches of data.
    for step, batch in enumerate(train_dataloader):
        # Move the batch tensors to the device.
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass through the model to get the outputs and calculate the loss.
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        # Calculate the accuracy for this batch by comparing predicted start and end positions with the true positions.
        start_positions = outputs.start_logits.argmax(dim=1)
        end_positions = outputs.end_logits.argmax(dim=1)
        correct_start = (start_positions == batch["start_positions"])
        correct_end = (end_positions == batch["end_positions"])
        total_correct += (correct_start & correct_end).sum().item()
        total_examples += len(correct_start)

        # Backpropagation and optimization step.
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        # Print the loss and accuracy at every nth step (e.g., every 10 steps).
        if step % 10 == 0:
            print(f"Step: {step}, Loss: {loss.item()}, Accuracy: {total_correct / total_examples:.2%}")

    # Print the average loss and accuracy for the epoch.
    average_loss = total_loss / len(train_dataloader)
    accuracy = total_correct / total_examples
    print(f"Epoch {epoch + 1} - Average Loss: {average_loss}, Accuracy: {accuracy:.2%}")

    # Evaluate the model after each epoch using the answer_question function.
    model.eval()
    with torch.no_grad():
        question = "What is the capital of France?"
        context = "France is a country in Europe. Its capital is Paris, which is known for its culture, history, and architecture."
        answer = answer_question(question, context, model, tokenizer, device)
        print(f"Epoch {epoch + 1} - Question: {question}")
        print(f"Epoch {epoch + 1} - Answer: {answer}")

# Print a message to indicate that the training is finished.
print("Training finished!")


Epoch: 1
Step: 0, Loss: 6.191780090332031, Accuracy: 0.00%
Step: 10, Loss: 4.131082534790039, Accuracy: 21.59%
Step: 20, Loss: 3.694505214691162, Accuracy: 28.57%
Step: 30, Loss: 3.7747769355773926, Accuracy: 26.21%
Step: 40, Loss: 4.038082122802734, Accuracy: 28.05%
Step: 50, Loss: 3.6977005004882812, Accuracy: 28.68%
Step: 60, Loss: 3.7537341117858887, Accuracy: 28.28%
Step: 70, Loss: 3.1831679344177246, Accuracy: 27.64%
Step: 80, Loss: 3.1349034309387207, Accuracy: 28.55%
Step: 90, Loss: 2.5715441703796387, Accuracy: 29.53%
Step: 100, Loss: 4.530526161193848, Accuracy: 29.95%
Step: 110, Loss: 4.310975551605225, Accuracy: 31.08%
Step: 120, Loss: 3.3954577445983887, Accuracy: 31.51%
Epoch 1 - Average Loss: 3.3789450368881226, Accuracy: 31.50%
Epoch 1 - Question: What is the capital of France?
Epoch 1 - Answer: [CLS]
Epoch: 2
Step: 0, Loss: 2.512270450592041, Accuracy: 37.50%
Step: 10, Loss: 3.4524338245391846, Accuracy: 32.95%
Step: 20, Loss: 2.2649569511413574, Accuracy: 33.33%
Step: